# Recommender Systems - Data Mining Lab 2

In [2]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

### Data loading

In [3]:
train_file_path = 'data/lab2_train.csv' 
test_file_path = 'data/lab2_test.csv'

train_data = pd.read_csv(train_file_path, delimiter=',')
test_data = pd.read_csv(test_file_path, delimiter=',')


In [62]:
#print(train_data)
like_train_data = train_data[train_data.is_like == True]
like_train_data = like_train_data.sort_values("user_to_id")
match_train_data = train_data[train_data.is_match ==True]
match_train_data = match_train_data.sort_values("user_from_id")

liked_user_count = like_train_data["user_to_id"].nunique()
user_count = like_train_data["user_from_id"].nunique()
match_count = match_train_data["user_from_id"].nunique()

print(user_count)
print(liked_user_count)
print(match_count)

print(train_data[train_data.user_to_id == 3349])
print(train_data[train_data.user_from_id == 3349])


#print(like_train_data)

print(match_train_data)

2107
1707
345
       user_from_id  user_to_id  is_like is_match
17270          1365        3349    False    False
41169          1918        3349    False    False
47470           804        3349    False    False
48542          1308        3349    False    False
50527          1000        3349    False    False
53983          1331        3349    False    False
55999          1001        3349    False    False
57098           779        3349    False    False
67910          1649        3349    False    False
74585          2001        3349    False    False
76220           995        3349    False    False
      user_from_id  user_to_id  is_like is_match
8137          3349        1359     True     True
       user_from_id  user_to_id  is_like is_match
62638             1        3046     True     True
40079             4        3058     True     True
676              19        2550     True     True
73991            36        3335     True     True
8940             44         150     Tr

## Familiarization
### Important properties of the data

### Types of people
There are only 1707 user who have one like or more of the 2107 users who gave likes. Of all of these users there are only 415 matches. Of these matches there are 345 unique users who receive a match. So there is only a small group of users who receive more than one match.

- Someone who receives a lot of likes but likes almost no one. User 2 receives 45 likes but likes none of them. Therefore not leading to a match

- Someone who receives and gives about only dislikes. For example user 28 is very active on the platform but gives about only dislikes and receives only dislikes. Therefore not leading to a match

- Someone who gives a lot of likes and receives only dislikes. For example user 18 gives 26 likes and receives no likes back and has thus not a match.

